<a href="https://colab.research.google.com/github/LukeWeidenwalker/fastai-notebooks/blob/master/08_movielens_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastai

     |████████████████████████████████| 204kB 8.3MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 


In [31]:
from fastai.collab import *
from fastai.tabular.all import *
import pandas as pd

In [3]:
path = untar_data(URLs.ML_100k)

In [ ]:
with open(path / "README") as readme:
  for row in readme:
    print(row)

In [24]:
raw_df = pd.read_csv(path/"u.data", delimiter="\t", names=["user_id", "item_id", "rating", "timestamp"])
raw_df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [23]:
movie_df = pd.read_csv(path/"u.item", delimiter="|", encoding="latin-1", usecols=(0,1), names=('movie','title'), header=None)
movie_df.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [29]:
ratings = raw_df.merge(movie_df, left_on="item_id", right_on="movie")
ratings.drop(["movie"], axis=1, inplace=True)
ratings.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [55]:
print(f"Number of Users: {len(dls.classes['user_id'])}")
print(f"Number of Movies: {len(dls.classes['title'])}")

Number of Users: 944
Number of Movies: 1665


In [32]:
dls = CollabDataLoaders.from_df(ratings, item_name="title", rating_name="rating", bs=64)

In [56]:
# (tensor(user_ids, movie_ids), tensor(ratings))
# shape=(64, 2)                 shape=(64, 1) 
dls.one_batch()[0].shape

torch.Size([64, 2])

In [ ]:
batch

In [61]:
batch = dls.one_batch()
one_hot(batch[0][1], len(dls.classes['user_id']))

IndexError: ignored

In [ ]:
class PMFModel(Module):
  def __init__(self, n_users, n_movies, n_user_factors, n_item_factors):
    self.user_factors = Embedding(n_users, n_user_factors)
    self.user_bias = nn.Parameter(torch.randn(size=(n_users, 1)))
    self.movie_factors = Embedding(n_movies, n_movie_factors)
    self.movie_bias = nn.Parameter(torch.randn(size=(n_movies, 1)))
  
  def forward(self, x):
    # shape of x: (shape=(64, 2), shape=(64, 1))
    

    pass

In [ ]:
learn = Learner(dls, PMFModel(), loss_func=MSELossFlat())